In [102]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, TensorDataset
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torchvision import transforms
from tqdm import tqdm


In [103]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
    print("Apple GPU")
elif torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA GPU")
else:
    device = torch.device('cpu')

Apple GPU


In [107]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.2860,), std=(0.3530,))
])


trainData = datasets.FashionMNIST(
    root="./data",
    train=True,
    transform=transform,
    download=True,
    target_transform=None
)

testData = datasets.FashionMNIST(
    root="./data",
    train=False,
    transform=transform,
    download=True,
    target_transform=None
)

print(trainData, testData,) #trainData.classes, trainData.data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.286,), std=(0.353,))
           ) Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.286,), std=(0.353,))
           )


In [108]:
trainData.classes = trainData.class_to_idx
testData.classes = testData.class_to_idx

In [109]:
trainData.data[0], trainData.targets[0]

(tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1,   0,
            0,  13,  73,   0,   0,   1,   4,   0,   0,   0,   0,   1,   1,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,   0,
           36, 136, 127,  62,  54,   0,   0,   0,   1,   3,   4,   0,   0,   3],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,   0,
          102, 204, 176, 134, 144, 123,  23,   0,   0,   0,   0,  12,  10,   0],
         [  0,   0,   0,   0,   0,

In [110]:
class CNNModel(nn.Module):

    def __init__(self, infeatures, hiddenfeatures, outfeatures):
        super().__init__()
        self.cnnModule1 = nn.Sequential(
            nn.Conv2d(in_channels=infeatures, out_channels=hiddenfeatures, kernel_size=2, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.cnnModule2 = nn.Sequential(
            nn.Conv2d(in_channels=hiddenfeatures, out_channels=hiddenfeatures, kernel_size=2, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features= 4 * hiddenfeatures, out_features=outfeatures)
        )
    
    def forward(self, x):
        x = self.cnnModule1(x)
        # print(x.shape)
        x = self.cnnModule2(x)
        # print(x.shape)
        x = self.classifier(x)
        return x
    
model = CNNModel(infeatures=1, hiddenfeatures=10, outfeatures=10)
# model.to(device)
test = torch.rand(1, 1, 28, 28)
model(test).shape

torch.Size([1, 10])

In [120]:
class FashionDataset(Dataset):
    def __init__(self, data, scale=10.0):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        X, y = self.data[idx]
        # print(X, y)


        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(y, dtype=torch.long)
        )
    

trainDataset = FashionDataset(trainData)
testDataset = FashionDataset(testData)

# trainDataset.__getitem__(0)

In [122]:
model = CNNModel(infeatures=1, hiddenfeatures=64, outfeatures=10)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01)
lossFn = CrossEntropyLoss()
epochs = 10


trainDataLoader = DataLoader(trainDataset, batch_size=64, shuffle=True)
testDataLoader = DataLoader(testDataset, batch_size=64, shuffle=False)



# print(len(trainDataLoader))
for each_epoch in range(epochs):
    model.train()

    loop = tqdm(trainDataLoader, desc=f"Epoch [{each_epoch+1}/{epochs}]")
    trainLoss = 0.0
    validLoss = 0.0
    for batchX, batchY in loop:
        # print(batchX.shape, batchY.shape)
        batchX = batchX.to(device)
        batchY = batchY.to(device)

        pred = model(batchX).to(device)
        pred = pred.squeeze()

        loss = lossFn(pred, batchY)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        trainLoss += loss.item()
    #     break
    # break


    with torch.no_grad():
        for batchX, batchY in testDataLoader:
            batchX = batchX.to(device)
            batchY = batchY.to(device)

            pred = model(batchX).to(device)
            pred = pred.squeeze()

            loss = lossFn(pred, batchY)

            validLoss += loss.item()
        #     break
        # break
    trainLoss = trainLoss/len(trainDataLoader)
    validLoss = validLoss/len(testDataLoader)

    
    loop.write(f" train Loss {trainLoss:.10f} | test loss {validLoss:.10f} ")

            




Epoch [1/10]:   0%|          | 0/938 [00:00<?, ?it/s]/var/folders/y8/q2s37ndx6tg3lpzp3vp8xk_r0000gn/T/ipykernel_13679/3139814318.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(X, dtype=torch.float32),
Epoch [1/10]: 100%|██████████| 938/938 [00:05<00:00, 164.11it/s]


 train Loss 0.5458778526 | test loss 0.4443813825 


Epoch [2/10]: 100%|██████████| 938/938 [00:05<00:00, 169.63it/s]


 train Loss 0.4294564952 | test loss 0.4309619782 


Epoch [3/10]: 100%|██████████| 938/938 [00:05<00:00, 161.60it/s]


...

 train Loss 0.3686160824 | test loss 0.4041291331 


Epoch [9/10]: 100%|██████████| 938/938 [00:05<00:00, 174.57it/s]


 train Loss 0.3658026204 | test loss 0.4593518131 


Epoch [10/10]: 100%|██████████| 938/938 [00:05<00:00, 183.34it/s]


 train Loss 0.3663259915 | test loss 0.4359558935 
